In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 샘플 데이터 생성
news_data = {
    "article_id": [1, 2, 3, 4, 5],
    "title": [
        "코로나19 백신 개발 진전",
        "올림픽 개최 논란",
        "주식 시장 급등",
        "새로운 AI 기술 발표",
        "기후 변화 대응 정책",
    ],
    "content": [
        "백신 개발이 진전을 보이고 있다...",
        "올림픽 개최에 대한 논란이 계속되고 있다...",
        "주식 시장이 사상 최고치를 기록했다...",
        "새로운 AI 기술이 산업계에 혁명을 일으킬 전망이다...",
        "기후 변화에 대응하기 위한 새로운 정책이 발표되었다...",
    ],
    "category": ["건강", "스포츠", "경제", "기술", "환경"],
}

news_df = pd.DataFrame(news_data)

# 간단한 사용자 행동 데이터 생성
user_behavior = {
    "user_id": [1, 1, 2, 2, 3],
    "article_id": [1, 3, 2, 4, 5],
    "interaction": [1, 1, 1, 1, 1],  # 1은 읽음을 나타냄
}

user_df = pd.DataFrame(user_behavior)

# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(news_df["content"])

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


# 콘텐츠 기반 추천 함수
def content_based_recommend(article_id, cosine_sim=cosine_sim):
    idx = news_df.index[news_df["article_id"] == article_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]  # 상위 3개 추천
    article_indices = [i[0] for i in sim_scores]
    return news_df["title"].iloc[article_indices].tolist()


# 사용자 기반 추천 함수
def user_based_recommend(user_id):
    user_articles = user_df[user_df["user_id"] == user_id]["article_id"].tolist()
    recommendations = []
    for article in user_articles:
        recommendations.extend(content_based_recommend(article))
    return list(set(recommendations))  # 중복 제거


# 실시간 트렌드 시뮬레이션 (실제로는 실시간 데이터를 사용해야 함)
def get_trending_articles():
    return (
        news_df.sort_values(by="article_id", ascending=False)["title"].head(2).tolist()
    )


# 최종 추천 함수
def get_recommendations(user_id):
    content_recs = user_based_recommend(user_id)
    trending_recs = get_trending_articles()
    final_recs = content_recs + trending_recs
    return list(dict.fromkeys(final_recs))  # 중복 제거 및 순서 유지


# 테스트
print(get_recommendations(1))

['올림픽 개최 논란', '새로운 AI 기술 발표', '코로나19 백신 개발 진전', '주식 시장 급등', '기후 변화 대응 정책']


['기후 변화 대응 정책', '새로운 AI 기술 발표']